In [1]:
import geopandas as gpd

/Users/hania/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import pyrosm

In [3]:
osm = pyrosm.OSM("newyorkcity.osm.pbf")
gdf_driving = osm.get_network(network_type="driving")
gdf_driving

,access,area,bicycle,bridge,busway,cycleway,est_width,foot,footway,highway,...,tunnel,turn,width,id,timestamp,version,tags,osm_type,geometry,length
0,None,None,yes,None,None,None,None,None,None,primary,...,None,None,None,5029221,0,-1,"{""cycleway:right"":""lane"",""hgv"":""destination"",""...",way,"MULTILINESTRING ((-73.98923 40.69098, -73.9891...",92.0
1,None,None,yes,None,None,lane,None,None,None,secondary,...,None,None,None,5668966,0,-1,"{""hgv"":""destination"",""name_1"":""West 106 Street...",way,"MULTILINESTRING ((-73.96042 40.79821, -73.9607...",66.0
2,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,5668968,0,-1,"{""alt_name"":""West 80 Street"",""tiger:cfcc"":""A41...",way,"MULTILINESTRING ((-73.98200 40.78559, -73.9808...",674.0
3,None,None,None,None,None,None,None,None,None,tertiary,...,None,None,None,5668973,0,-1,"{""lane_markings"":""no"",""tiger:cfcc"":""A41"",""tige...",way,"MULTILINESTRING ((-73.93144 40.85908, -73.9312...",706.0
4,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,5668977,0,-1,"{""alt_name"":""Edgar Allen Poe Street"",""name_1"":...",way,"MULTILINESTRING ((-73.98107 40.78849, -73.9810...",762.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234247,None,None,None,None,None,None,None,None,None,service,...,None,None,None,1062720536,0,-1,None,way,"MULTILINESTRING ((-73.70004 40.84406, -73.7000...",28.0
234248,None,None,None,None,None,None,None,None,None,tertiary_link,...,None,None,None,1062720540,0,-1,None,way,"MULTILINESTRING ((-73.69058 40.84329, -73.6902...",38.0
234249,None,None,None,None,None,None,None,None,None,primary,...,None,None,None,1062720542,0,-1,"{""name_1"":""State Highway 101"",""old_ref"":""CR 10...",way,"MULTILINESTRING ((-73.69013 40.84328, -73.6901...",3.0
234250,None,None,None,None,None,None,None,None,None,service,...,None,None,None,1062720543,0,-1,None,way,"MULTILINESTRING ((-73.69416 40.83985, -73.6939...",16.0


In [4]:
gdf_zones = gpd.read_file('NYC_Taxi_Zones.geojson')
gdf_zones

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."
...,...,...,...,...,...,...,...
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,"MULTIPOLYGON (((-73.95834 40.71331, -73.95681 ..."
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,"MULTIPOLYGON (((-73.85107 40.91037, -73.85207 ..."
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,"MULTIPOLYGON (((-73.90175 40.76078, -73.90147 ..."
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,"MULTIPOLYGON (((-74.01333 40.70503, -74.01327 ..."


In [5]:
gdf_driving.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
gdf_zones.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# Clip roads to zones
gdf_rodes = gpd.clip(gdf_driving, gdf_zones)

In [8]:
gdf_rodes

,access,area,bicycle,bridge,busway,cycleway,est_width,foot,footway,highway,...,tunnel,turn,width,id,timestamp,version,tags,osm_type,geometry,length
11305,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,5711604,0,-1,"{""tiger:cfcc"":""A41"",""tiger:county"":""Richmond, ...",way,"MULTILINESTRING ((-74.23651 40.50043, -74.2365...",149.0
160568,None,None,None,None,None,None,None,None,None,service,...,None,None,None,738666464,0,-1,None,way,"LINESTRING (-74.23767 40.50015, -74.23805 40.5...",34.0
11857,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,5712589,0,-1,"{""tiger:cfcc"":""A41"",""tiger:county"":""Richmond, ...",way,"MULTILINESTRING ((-74.23774 40.50061, -74.2377...",98.0
13429,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,5715446,0,-1,"{""tiger:cfcc"":""A41"",""tiger:county"":""Richmond, ...",way,"MULTILINESTRING ((-74.23795 40.50125, -74.2379...",74.0
11858,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,5712590,0,-1,"{""name_1"":""Seabreeze Lane"",""tiger:cfcc"":""A41"",...",way,"MULTILINESTRING ((-74.23795 40.50125, -74.2379...",38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143318,None,None,None,None,None,None,None,None,None,service,...,None,None,None,658544098,0,-1,None,way,"LINESTRING (-73.90940 40.91486, -73.90952 40.9...",34.0
66425,None,None,None,None,None,None,None,None,None,service,...,None,None,None,162599511,0,-1,None,way,"MULTILINESTRING ((-73.90975 40.91463, -73.9095...",99.0
40940,None,None,None,None,None,None,None,None,None,residential,...,None,None,None,23469174,0,-1,"{""tiger:cfcc"":""A41"",""tiger:county"":""Westcheste...",way,"MULTILINESTRING ((-73.90952 40.91490, -73.9093...",79.0
82583,None,None,None,None,None,None,None,None,None,service,...,None,None,None,277182906,0,-1,None,way,"MULTILINESTRING ((-73.90919 40.91510, -73.9091...",11.0


In [14]:
roads_in_zones = gpd.sjoin(
    gdf_rodes,
    gdf_zones,
    predicate='intersects'
)

In [10]:
roads_in_zones

,access,area,bicycle,bridge,busway,cycleway,est_width,foot,footway,highway,...,osm_type,geometry,length,index_right,shape_area,objectid,shape_leng,location_id,zone,borough
11305,None,None,None,None,None,None,None,None,None,residential,...,way,"MULTILINESTRING ((-74.23651 40.50043, -74.2365...",149.0,45,0.00194465649192,44,0.235688967594,44,Charleston/Tottenville,Staten Island
160568,None,None,None,None,None,None,None,None,None,service,...,way,"LINESTRING (-74.23767 40.50015, -74.23805 40.5...",34.0,45,0.00194465649192,44,0.235688967594,44,Charleston/Tottenville,Staten Island
11857,None,None,None,None,None,None,None,None,None,residential,...,way,"MULTILINESTRING ((-74.23774 40.50061, -74.2377...",98.0,45,0.00194465649192,44,0.235688967594,44,Charleston/Tottenville,Staten Island
13429,None,None,None,None,None,None,None,None,None,residential,...,way,"MULTILINESTRING ((-74.23795 40.50125, -74.2379...",74.0,45,0.00194465649192,44,0.235688967594,44,Charleston/Tottenville,Staten Island
11858,None,None,None,None,None,None,None,None,None,residential,...,way,"MULTILINESTRING ((-74.23795 40.50125, -74.2379...",38.0,45,0.00194465649192,44,0.235688967594,44,Charleston/Tottenville,Staten Island
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4786,None,None,None,None,None,None,None,None,None,residential,...,way,"MULTILINESTRING ((-73.91074 40.91286, -73.9107...",252.0,202,0.000744643168558,200,0.112661735435,200,Riverdale/North Riverdale/Fieldston,Bronx
143318,None,None,None,None,None,None,None,None,None,service,...,way,"LINESTRING (-73.90940 40.91486, -73.90952 40.9...",34.0,202,0.000744643168558,200,0.112661735435,200,Riverdale/North Riverdale/Fieldston,Bronx
66425,None,None,None,None,None,None,None,None,None,service,...,way,"MULTILINESTRING ((-73.90975 40.91463, -73.9095...",99.0,202,0.000744643168558,200,0.112661735435,200,Riverdale/North Riverdale/Fieldston,Bronx
40940,None,None,None,None,None,None,None,None,None,residential,...,way,"MULTILINESTRING ((-73.90952 40.91490, -73.9093...",79.0,202,0.000744643168558,200,0.112661735435,200,Riverdale/North Riverdale/Fieldston,Bronx


In [19]:
# Sum existing columns
roads_in_zones.set_crs(epsg=4326, inplace=True)
road_lenghts=roads_in_zones.groupby('index_right').sum()
#roads_in_zones
roads_in_zones['rdlength'] = roads_in_zones['length']
sub = roads_in_zones[['rdlength', 'index_right', 'zone']].groupby('index_right').sum()
sub

,rdlength
index_right,
0,84414.0
1,12534.0
2,73892.0
3,17180.0
4,78034.0
...,...
258,51799.0
259,92666.0
260,103958.0
